# P2WPKH Transactions - BX Example

<br>
<img src="images/p2wpkh.jpg" alt="drawing" style="" width="800px"/>

<hr style="border: 0.5px dashed #000;">



In [ ]:
// Compiler & linker information for c++ interpreter.
#pragma cling add_include_path("/usr/local/include")
#pragma cling add_library_path("/usr/local/lib")
#pragma cling load("bitcoin","secp256k1","pthread","boost_chrono-mt","boost_date_time-mt","boost_filesystem","boost_iostreams-mt","boost_locale-mt","boost_log-mt","boost_program_options-mt","boost_regex-mt","boost_system","boost_thread-mt")

// Libbitcoin-System.
#include <bitcoin/bitcoin.hpp> 



## 1. Spend to a P2WPKH output.


* Output script: **`zero [20-byte hash160(public key)]`**

In [ ]:
{
    //**************************************************************
    // 1. Restore wallet and generate spending / receiving key pairs
    //**************************************************************

    std::string my_sentence = "amount right cheese defy click eight slight strategy replace earn simple labor";
    const auto my_word_list = bc::split(my_sentence, " ", true);

    const auto hd_seed = bc::wallet::decode_mnemonic(my_word_list);

    bc::wallet::hd_private hd_master_private(bc::to_chunk(hd_seed),
        bc::wallet::hd_private::testnet);
    const auto hd_master_public = hd_master_private.to_public();

    // We will be spending a p2pkh output script controlled by
    // the following key: m/44'/1'/1'/0/8
    //-------------------------------------------------------------

    const auto hd_m_44h_1h_1h_0_8 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(0)
        .derive_private(8);
    const auto secret_44h_1h_1h_0_8 = hd_m_44h_1h_1h_0_8.secret();
    const auto pubkey_44h_1h_1h_0_8 =
        hd_m_44h_1h_1h_0_8.to_public().point();

    bc::wallet::payment_address payment_address_44h_1h_1h_0_8(
        pubkey_44h_1h_1h_0_8, bc::wallet::payment_address::testnet_p2kh);

    // Generate receiving keys for the P2WPKH output.
    //-------------------------------------------------------------

    // Receiving Keys in 3rd wallet account.
    const auto hd_m_44h_1h_2h_0_8 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(2 + bc::wallet::hd_first_hardened_key)
        .derive_private(0)
        .derive_private(8);
    const auto pubkey_44h_1h_2h_0_8 =
        hd_m_44h_1h_2h_0_8.to_public().point();

    //**************************************************************
    // 2. Fetch previous P2PKH UTXO which we will spend.
    //**************************************************************

    // Previous TX hash.
    const auto prev_tx_literal =
        "86ad635645920497c233cc33556463fa6258ab8f9c5f7d12748a6638216e3363";
    bc::hash_digest prev_tx_hash;
    bc::decode_hash(prev_tx_hash, prev_tx_literal);

    // Previous UXTO prev_index.
    uint32_t prev_index = 8;
    bc::chain::output_point uxto_to_spend(prev_tx_hash, prev_index);

    // Previous  script.
    const auto prev_p2pkh_output_script =
        bc::chain::script::to_pay_key_hash_pattern(
            bc::bitcoin_short_hash(pubkey_44h_1h_1h_0_8));

    //**************************************************************
    // 3. Build, Sign & Complete Transaction.
    //**************************************************************

    // A. Encode P2WPKH output.
    //-------------------------------------------------------------

    // P2WPKH output script.
    //  0 [20-byte hash160(public key)]
    bc::machine::operation::list p2wpkh_operations;
    p2wpkh_operations.push_back(
        bc::machine::operation(bc::machine::opcode::push_size_0));
    p2wpkh_operations.push_back(
        bc::machine::operation(bc::to_chunk(bc::bitcoin_short_hash(
          pubkey_44h_1h_2h_0_8))));

    // Compute fees.
    // tx bytes: 4 + 1 + 1*(32+4+110+4) + 1 + 1*(8+1+1+22) + 4 = 192
    // 1 non-witness sigop: 1 * 400
    // 499543 - 1 * 400 - 192 = 498951
    uint64_t output_amount(498951);

    // Build P2WPKH output.
    bc::chain::output p2wpkh_output(output_amount, p2wpkh_operations);

    // B. Build Transaction for signing.
    //-------------------------------------------------------------

    // Build input0 object.
    bc::chain::input input0;
    input0.set_previous_output(uxto_to_spend);
    input0.set_sequence(bc::max_input_sequence);

    // Build Transaction.
    bc::chain::transaction tx;
    tx.set_version(1u);
    tx.inputs().push_back(input0);
    tx.outputs().push_back(p2wpkh_output);

    // Signature.
    bc::endorsement signature;
    uint8_t input0_index(0u);
    bc::chain::script::create_endorsement(signature, secret_44h_1h_1h_0_8,
      prev_p2pkh_output_script, tx, input0_index,
      bc::machine::sighash_algorithm::all);

    // B. Finalise Transaction.
    //-------------------------------------------------------------

    // Set input script into transaction.
    bc::machine::operation::list p2pkh_operations;
    p2pkh_operations.push_back(bc::machine::operation(signature));
    p2pkh_operations.push_back(bc::machine::operation(
        bc::to_chunk(pubkey_44h_1h_1h_0_8)));
    bc::chain::script p2pkh_input_script(p2pkh_operations);
    tx.inputs()[0].set_script(p2pkh_input_script);

    std::cout << bc::encode_base16(tx.to_data()) << std::endl;
}

In [ ]:
!bx tx-decode -f json 010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86080000006b48304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e012103aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67ffffffff01079d070000000000160014757d9e1576e8b890abdeba9971f7238ab9aa0a8a00000000 
    

In [ ]:
!bx validate-tx 010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86080000006b48304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e012103aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67ffffffff01079d070000000000160014757d9e1576e8b890abdeba9971f7238ab9aa0a8a00000000
    

In [ ]:
!bx bitcoin256 010000000163336e2138668a74127d5f9c8fab5862fa63645533cc33c2970492455663ad86080000006b48304502210093fee9cea2af3245f50f656fb29cef95faed4945b3a2b9fff841ce14cbd9d33402205d56f641944b62d10bb803e64384e7507f31999aba38e4e85c71892e98e2819e012103aab4421bb35035cc193ee4489039df8cb5a9b418a2f1d30e312f7b2cf59f6d67ffffffff01079d070000000000160014757d9e1576e8b890abdeba9971f7238ab9aa0a8a00000000
    

<hr style="border: 0.5px dashed #000;">

## 2. Spend a P2WPKH output.

* Input script: **`empty`**
* Script code: **`DUP HASH160 [20-byte hash160(Public Key)] EQUALVERIFY CHECKSIG`**
* Witness: **`[Signature] [Public Key]`**

In [ ]:
{
    //**************************************************************************
    // 1. Restore wallet and generate spending key pairs
    //**************************************************************************

    std::string my_sentence = "amount right cheese defy click eight slight strategy replace earn simple labor";
    const auto my_word_list = bc::split(my_sentence, " ", true);

    const auto hd_seed = bc::wallet::decode_mnemonic(my_word_list);

    bc::wallet::hd_private hd_master_private(bc::to_chunk(hd_seed),
        bc::wallet::hd_private::testnet);
    const auto hd_master_public = hd_master_private.to_public();


    // We will be spending a p2wpkh output script controlled by
    // the following key: m/44'/1'/2'/0/8
    //--------------------------------------------------------------------------

    const auto hd_m_44h_1h_2h_0_8 = hd_master_private
        .derive_private(44 + bc::wallet::hd_first_hardened_key)
        .derive_private(1 + bc::wallet::hd_first_hardened_key)
        .derive_private(2 + bc::wallet::hd_first_hardened_key)
        .derive_private(0)
        .derive_private(8);
    const auto secret_44h_1h_2h_0_8 = hd_m_44h_1h_2h_0_8.secret();
    const auto pubkey_44h_1h_2h_0_8 =
        hd_m_44h_1h_2h_0_8.to_public().point();

    //**************************************************************************
    // 2. Fetch previous P2PKH UTXO which we will spend.
    //**************************************************************************

    // Previous TX hash.
    const auto prev_tx_literal =
        "9568c5a759f53c3db476362bdb4e36db31f4986fc2189cab36e9880ef2b3424b";
    bc::hash_digest prev_tx_hash;
    bc::decode_hash(prev_tx_hash, prev_tx_literal);

    // Previous UXTO prev_index.
    uint32_t prev_index = 0;
    bc::chain::output_point uxto_to_spend(prev_tx_hash, prev_index);

    // Previous output script not required for signing witness transactions.

    // Previous output amount.
    uint64_t previous_output_amount(498951);

    //**************************************************************************
    // 3. Build, Sign & Complete Transaction.
    //**************************************************************************

    // A. Encode P2PKH output.
    //--------------------------------------------------------------------------

    // We will send our funds to n2MBcctgzBt1h8Nvfu3XAEPJLrmWET7emw
    // ...pubkeyhash: e48199d47742b245464b1366d95ef26aa4c8bb2c

    const auto receiving_pubkeyhash =
        bc::base16_literal("e48199d47742b245464b1366d95ef26aa4c8bb2c");
    const auto p2pkh_operations =
        bc::chain::script::to_pay_key_hash_pattern(receiving_pubkeyhash);

    // Compute fees.
    //--------------------------------------------------------------------------

    // tx bytes: 4 + 1 + 1*(32+4+0+4) + 1 + 1*(8+33) + 4 = 91

    // Witness tx size: (91) + 72 + 33 = 196

    // # of sigops: one in script_code, one in output.
    // Output amount = 498951 - 1 * 100 - 1 * 400 - 196 * 1 = 498254

    uint64_t output_amount(498254);

    // Build P2WPKH output.
    bc::chain::output p2wpkh_output(output_amount, p2pkh_operations);

    // B. Build & Sign Transaction.
    //--------------------------------------------------------------------------

    // Build input0 object.
    bc::chain::input input;
    input.set_previous_output(uxto_to_spend);
    input.set_sequence(bc::max_input_sequence);

    // Build Transaction.
    bc::chain::transaction tx;
    tx.set_version(1u);
    tx.inputs().push_back(input);
    tx.outputs().push_back(p2wpkh_output);

    // Script code.
    const auto p2wpkh_script_code = bc::chain::script::to_pay_key_hash_pattern(
        bc::bitcoin_short_hash(pubkey_44h_1h_2h_0_8));

    // Previous input index.
    uint8_t input_index(0);

    // Sign transaction.
    bc::endorsement signature;
    bc::chain::script::create_endorsement(signature, secret_44h_1h_2h_0_8,
        p2wpkh_script_code, tx, input_index, bc::machine::sighash_algorithm::all,
        bc::machine::script_version::zero, previous_output_amount);

    // C. Finalise Transaction.
    //--------------------------------------------------------------------------

    // Build witness (...instead of input script)
    // [signature] [publicKey]
    bc::data_stack witness_stack;
    witness_stack.push_back(signature);
    witness_stack.push_back(bc::to_chunk(pubkey_44h_1h_2h_0_8));
    bc::chain::witness p2wpkh_witness(witness_stack);
    tx.inputs()[0].set_witness(p2wpkh_witness);

    // Serialisation to include witness: wire=true/witness=true
    std::cout << bc::encode_base16(tx.to_data(true, true)) << std::endl;
}

In [ ]:
!bx tx-decode -f json 010000000001014b42b3f20e88e936ab9c18c26f98f431db364edb2b3676b43d3cf559a7c568950000000000ffffffff014e9a0700000000001976a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac0247304402205b937aea90032d4e79c03f0ede44f24beecac30265e61d9dab6da6ed98bd0c3d0220476f7fae25d1cbbe1363351ecd92a2450aa86fa4a1a23b44e0fb79bb4e872ecf012102adbc4998becd5da552e6763a7007ab0a011c6d147bda61b161371de65da9aae200000000

In [ ]:
!bx validate-tx 010000000001014b42b3f20e88e936ab9c18c26f98f431db364edb2b3676b43d3cf559a7c568950000000000ffffffff014e9a0700000000001976a914e48199d47742b245464b1366d95ef26aa4c8bb2c88ac0247304402205b937aea90032d4e79c03f0ede44f24beecac30265e61d9dab6da6ed98bd0c3d0220476f7fae25d1cbbe1363351ecd92a2450aa86fa4a1a23b44e0fb79bb4e872ecf012102adbc4998becd5da552e6763a7007ab0a011c6d147bda61b161371de65da9aae200000000
    